# HPE Container Platform (HPE CP) API series  - Clean Up of leftover resources in K8s cluster


#### Initialize the environment:

In [ ]:
#
# environment variables to be adjusted/verified by the student
#
student="hpedevadmin" # your Jupyter Notebook student Identifier (i.e.: student<xx>)
username="demoadmin" # your HPE CP tenant login credentials - username 
password="CleanUpDISCO2020" # your HPE CP tenant login credentials - password
#
# fixed environment variables setup by the HPE CP lab administrator
#
controller_endpoint="gateway1.hpedevlab.net:8080"
gateway_host="gateway1.hpedevlab.net"
tenantname="K8sHackTenant"
k8sClusterId="1"  #this is the K8s Cluster Id provided by the HPE CP admisnistrator and assigned to your K8s tenant.
helloWorldApp="hello-world-app.yaml" # the application manifest you will deploy in this lab
tensorFlowApp="tensorflow-notebook-config-cluster.yaml" # the kubedirector application cluster configuration TensorFlow

echo "your operation context is:" $username "on tenant" $tenantname 

#### Authenticate as a Tenant admin in the specified tenant:

In [ ]:
sessionlocation=$(curl -k -i -s --request POST "https://${controller_endpoint}/api/v2/session" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
"name": "'"$username"'",
"password": "'"$password"'",
"tenant_name": "'"$tenantname"'"
}' | grep Location | awk '{print $2}' | tr -d '\r') #we remove any cr that might exist
echo "This is your session location: " $sessionlocation
SessionId=$(echo $sessionlocation | cut -d'/' -f 5) # extract sessionId for later, for logout
echo "This is your session_Id:" $SessionId

#### Get the Kubeconfig file for your tenant working context:

In [ ]:
curl -k -s --request GET "https://${controller_endpoint}/api/v2/k8skubeconfig" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' > ${student}_kubeconfig

In [ ]:
cat ${student}_kubeconfig

> Notice the kubeconfig file includes the valid token (session location) for the current session you previously established.

#### Define the Kubeconfig file as a shell environment variable:

In [ ]:
#define the Kubeconfig file as a shell environment variable to tell kubectl where to look for the kubeconfig file
export KUBECONFIG="${student}_kubeconfig"
echo $KUBECONFIG

#### Check your working tenant context:

In [ ]:
kubectl config current-context

You can now send Kubernetes API requests using a K8s API client such as **kubectl** to cleanup leftover K8s resources deployed by attendees.

## Deleting Hello-world and tensorFlow applications

#### -1- Delete Hello World containerized applications:

Verifying if there is any hello-world app resources. A result of `: 1` means no PODs left for hello-world apps.

In [ ]:
kubectl get pod | grep hello-world

Depending on the output of the command, you may want to delete leftover K8s resources for hello-world applications.

In [ ]:
for i in {1..75}
do
   echo "Cleanup student$i - Hello-world app"
   sed -i "s/example/student$i/g" $helloWorldApp
   #cat $helloWorldApp
   kubectl delete -f $helloWorldApp
   echo "reset YAML file after cleanup of student$i"
   sed -i "s/student$i/example/g" $helloWorldApp
   #cat $helloWorldApp
done

In [ ]:
kubectl get pod | grep hello-world

#### -2- Delete kubedirectorapp Tensorflow instances:

Verifying if there is any kubedirectorapp instances.

In [ ]:
kubectl get kubedirectorcluster | grep tensorflow

Depending on the output of the command, you may want to delete leftover K8s resources for TensorFlow applications.

In [ ]:
for i in {1..75}
do
   echo "Cleanup student$i - TensorFlow app"
   sed -i "s/example/student$i/g" $tensorFlowApp
   #cat $tensorFlowApp
   kubectl delete -f $tensorFlowApp
   echo "reset YAML file after cleanup of student$i"
   sed -i "s/student$i/example/g" $tensorFlowApp
   #cat $tensorFlowApp
done

In [ ]:
kubectl get kubedirectorcluster | grep tensorflow

Close the login session

In [ ]:
curl -k -i -s --request DELETE "https://${controller_endpoint}/api/v2/session/${SessionId}" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json'

The status *204 No Content* means the session object has been deleted.

Delete the kubeconfig file

In [ ]:
rm ${student}_kubeconfig